In [26]:
#| default_exp preprocessing

# Preprocessing
> Utilities for processing data before training/analysis

In [1]:
#| export
import warnings
from datetime import date, datetime
from functools import partial
from typing import Union

import numpy as np
import pandas as pd

from utilsforecast.compat import DFType, pl, pl_DataFrame, pl_Series
from utilsforecast.processing import group_by, repeat
from utilsforecast.validation import _is_int_dtype, validate_format, validate_freq

In [2]:
#| hide
from nbdev import show_doc

from utilsforecast.data import generate_series

In [3]:
#| exporti
def _determine_bound(bound, freq, times_by_id, agg) -> np.ndarray:
    if bound == 'per_serie':
        out = times_by_id[agg].to_numpy()
    else:
        # the following return a scalar
        if bound == 'global':
            val = getattr(times_by_id[agg].to_numpy(), agg)()
            if isinstance(freq, str):
                val = np.datetime64(val)
        else:
            if isinstance(freq, str):
                # this raises a nice error message if it isn't a valid datetime
                if isinstance(bound, pd.Timestamp) and bound.tz is not None:
                    bound = bound.tz_convert('UTC').tz_localize(None)
                val = np.datetime64(bound)
            else:
                val = bound
        out = np.full(times_by_id.shape[0], val)
    if isinstance(freq, str):
        out = out.astype(f'datetime64[{freq}]')
    return out

In [4]:
#| exporti
def _determine_bound_pl(
    bound: Union[str, int, date, datetime],
    times_by_id: pl_DataFrame,
    agg: str,
) -> pl_Series:
    if bound == 'per_serie':
        out = times_by_id[agg]
    else:
        if bound == 'global':
            val = getattr(times_by_id[agg], agg)()
        else:
            val = bound
        out = repeat(pl_Series([val]), times_by_id.shape[0])
    return out

In [5]:
#| export
def id_time_grid(
    df: DFType,
    freq: Union[str, int],
    start: Union[str, int, date, datetime] = 'per_serie',
    end: Union[str, int, date, datetime] = 'global',
    id_col: str = 'unique_id',
    time_col: str = 'ds',
) -> DFType:
    """Generate all expected combiations of ids and times.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input data
    freq : str or int
        Series' frequency
    start : str, int, date or datetime.
        Initial timestamp for the series.
            * 'per_serie' uses each serie's first timestamp
            * 'global' uses the first timestamp seen in the data
            * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1)
    end : str, int, date or datetime.
        Initial timestamp for the series.
            * 'per_serie' uses each serie's last timestamp
            * 'global' uses the last timestamp seen in the data
            * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1)
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestamp.

    Returns
    -------
    pandas or polars DataFrame
        Dataframe with expected ids and times.
    """
    if isinstance(df, pl_DataFrame):
        times_by_id = group_by(df, id_col).agg(
            pl.col(time_col).min().alias('min'),
            pl.col(time_col).max().alias('max'),
        ).sort(id_col)
        starts = _determine_bound_pl(start, times_by_id, 'min')
        ends = _determine_bound_pl(end, times_by_id, 'max')
        grid = pl_DataFrame({id_col: times_by_id[id_col]})
        if _is_int_dtype(starts):
            grid = grid.with_columns(
                pl.int_ranges(starts, ends + freq, step=freq, eager=True).alias(time_col)
            )
        else:
            if starts.dtype == pl.Date:
                ranges_fn = pl.date_ranges
            else:
                ranges_fn = partial(
                    pl.datetime_ranges,
                    time_unit=df.schema[time_col].time_unit,
                )
            grid = grid.with_columns(
                ranges_fn(
                    starts,
                    ends,
                    interval=freq,
                    eager=True,
                ).alias(time_col)
            )
        return grid.explode(time_col)
    if isinstance(freq, str):
        offset = pd.tseries.frequencies.to_offset(freq)
        n = offset.n
        if isinstance(offset.base, pd.offsets.Minute):
            # minutes are represented as 'm' in numpy
            freq = 'm'
        elif isinstance(offset.base, pd.offsets.BusinessDay):
            if n != 1:
                raise NotImplementedError('Multiple of a business day')
            freq = 'D'
        elif isinstance(offset.base, pd.offsets.Hour):
            # hours are represented as 'h' in numpy
            freq = 'h'
        elif isinstance(offset.base, (pd.offsets.QuarterBegin, pd.offsets.QuarterEnd)):
            n = 3
            freq = 'M'
        elif isinstance(offset.base, (pd.offsets.YearBegin, pd.offsets.YearEnd)):
            freq = 'Y'
        elif isinstance(offset.base, pd.offsets.Second):
            freq = "s"                    
        elif isinstance(offset.base, pd.offsets.Milli):
            freq = "ms"
        elif isinstance(offset.base, pd.offsets.Micro):
            freq = "us"
        elif isinstance(offset.base, pd.offsets.Nano):
            freq = "ns"
        if n > 1:
            freq = freq.replace(str(n), '')
        try:
            pd.Timedelta(offset)
        except ValueError:
            # irregular freq, try using first letter of abbreviation
            # such as MS = 'Month Start' -> 'M', YS = 'Year Start' -> 'Y'
            freq = freq[0]
        delta: Union[np.timedelta64, int] = np.timedelta64(n, freq)
        if df[time_col].dt.tz is not None:
            df = df.copy(deep=False)
            df[time_col] = df[time_col].dt.tz_convert('UTC').dt.tz_localize(None)
    else:
        delta = freq
    times_by_id = df.groupby(id_col, observed=True)[time_col].agg(['min', 'max'])
    starts = _determine_bound(start, freq, times_by_id, 'min')
    ends = _determine_bound(end, freq, times_by_id, 'max') + delta
    sizes = ((ends - starts) / delta).astype(np.int64)
    times = np.hstack(
        [
            np.arange(start, end, delta) for start, end in zip(starts, ends)
        ]
    )
    uids = np.repeat(times_by_id.index, sizes)    
    if isinstance(freq, str):
        if isinstance(offset.base, pd.offsets.BusinessDay):
            # data was generated daily, we need to keep only business days 
            bdays = np.is_busday(times)
            uids = uids[bdays]
            times = times[bdays]
        times = pd.Index(times.astype('datetime64[ns]', copy=False))
        first_time = np.datetime64(df.iloc[0][time_col])
        was_truncated = first_time != first_time.astype(f'datetime64[{freq}]')
        if was_truncated:
            times += offset.base
    return pd.DataFrame(
        {
            id_col: uids,
            time_col: times,
        }
    )

In [6]:
#| export
def fill_gaps(
    df: DFType,
    freq: Union[str, int],
    start: Union[str, int, date, datetime] = 'per_serie',
    end: Union[str, int, date, datetime] = 'global',
    id_col: str = 'unique_id',
    time_col: str = 'ds',
) -> DFType:
    """Enforce start and end datetimes for dataframe.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input data
    freq : str or int
        Series' frequency
    start : str, int, date or datetime.
        Initial timestamp for the series.
            * 'per_serie' uses each serie's first timestamp
            * 'global' uses the first timestamp seen in the data
            * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1)
    end : str, int, date or datetime.
        Initial timestamp for the series.
            * 'per_serie' uses each serie's last timestamp
            * 'global' uses the last timestamp seen in the data
            * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1)
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestamp.

    Returns
    -------
    filled_df : pandas or polars DataFrame
        Dataframe with gaps filled.
    """
    validate_format(df, id_col=id_col, time_col=time_col, target_col=None)
    validate_freq(df[time_col], freq=freq)
    
    grid = id_time_grid(
        df=df,
        freq=freq,
        start=start,
        end=end,
        id_col=id_col,
        time_col=time_col,
    )
    if isinstance(df, pl_DataFrame):
        return grid.join(df, on=[id_col, time_col], how='left')
    idx = pd.MultiIndex.from_frame(grid)
    if isinstance(freq, str):
        tz = df[time_col].dt.tz
        if tz is not None:
            df = df.copy(deep=False)
            df[time_col] = df[time_col].dt.tz_convert('UTC').dt.tz_localize(None)
    res = df.set_index([id_col, time_col]).reindex(idx).reset_index()
    if isinstance(freq, str):
        if tz is not None:
            res[time_col] = res[time_col].dt.tz_localize('UTC').dt.tz_convert(tz)
    extra_cols = df.columns.drop([id_col, time_col]).tolist()
    if extra_cols:
        check_col = extra_cols[0]
        if res[check_col].count() < df[check_col].count():
            warnings.warn(
                "Some values were lost during filling, "
                "please make sure that all your times meet the specified frequency.\n"
                "For example if you have 'W-TUE' as your frequency, "
                "make sure that all your times are actually Tuesdays."
            )
    return res

In [7]:
show_doc(fill_gaps)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/preprocessing.py#L197){target="_blank" style="float:right; font-size:smaller"}

### fill_gaps

>      fill_gaps (df:~DFType, freq:Union[str,int],
>                 start:Union[str,int,datetime.date,datetime.datetime]='per_seri
>                 e',
>                 end:Union[str,int,datetime.date,datetime.datetime]='global',
>                 id_col:str='unique_id', time_col:str='ds')

*Enforce start and end datetimes for dataframe.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input data |
| freq | Union |  | Series' frequency |
| start | Union | per_serie | Initial timestamp for the series.<br>    * 'per_serie' uses each serie's first timestamp<br>    * 'global' uses the first timestamp seen in the data<br>    * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1) |
| end | Union | global | Initial timestamp for the series.<br>    * 'per_serie' uses each serie's last timestamp<br>    * 'global' uses the last timestamp seen in the data<br>    * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1) |
| id_col | str | unique_id | Column that identifies each serie. |
| time_col | str | ds | Column that identifies each timestamp. |
| **Returns** | **DFType** |  | **Dataframe with gaps filled.** |

In [8]:
df = pd.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': pd.to_datetime(['2020', '2021', '2023', '2021', '2022']),
        'y': np.arange(5),
    }
)
df

,unique_id,ds,y
0,0,2020-01-01,0
1,0,2021-01-01,1
2,0,2023-01-01,2
3,1,2021-01-01,3
4,1,2022-01-01,4


The default functionality is taking the current starts and only extending the end date to be the same for all series.

In [9]:
fill_gaps(
    df,
    freq='YS',
)

,unique_id,ds,y
0,0,2020-01-01,0.0
1,0,2021-01-01,1.0
2,0,2022-01-01,NaN
3,0,2023-01-01,2.0
4,1,2021-01-01,3.0
5,1,2022-01-01,4.0
6,1,2023-01-01,NaN


We can also specify `end='per_serie'` to only fill possible gaps within each serie.

In [10]:
fill_gaps(
    df,
    freq='YS',
    end='per_serie',
)

,unique_id,ds,y
0,0,2020-01-01,0.0
1,0,2021-01-01,1.0
2,0,2022-01-01,NaN
3,0,2023-01-01,2.0
4,1,2021-01-01,3.0
5,1,2022-01-01,4.0


We can also specify an end date in the future.

In [11]:
fill_gaps(
    df,
    freq='YS',
    end='2024',
)

,unique_id,ds,y
0,0,2020-01-01,0.0
1,0,2021-01-01,1.0
2,0,2022-01-01,NaN
3,0,2023-01-01,2.0
4,0,2024-01-01,NaN
5,1,2021-01-01,3.0
6,1,2022-01-01,4.0
7,1,2023-01-01,NaN
8,1,2024-01-01,NaN


We can set all series to start at the same time.

In [12]:
fill_gaps(
    df,
    freq='YS',
    start='global'
)

,unique_id,ds,y
0,0,2020-01-01,0.0
1,0,2021-01-01,1.0
2,0,2022-01-01,NaN
3,0,2023-01-01,2.0
4,1,2020-01-01,NaN
5,1,2021-01-01,3.0
6,1,2022-01-01,4.0
7,1,2023-01-01,NaN


We can also set a common start date for all series (which can be earlier than their current starts).

In [13]:
fill_gaps(
    df,
    freq='YS',
    start='2019',
)

,unique_id,ds,y
0,0,2019-01-01,NaN
1,0,2020-01-01,0.0
2,0,2021-01-01,1.0
3,0,2022-01-01,NaN
4,0,2023-01-01,2.0
5,1,2019-01-01,NaN
6,1,2020-01-01,NaN
7,1,2021-01-01,3.0
8,1,2022-01-01,4.0
9,1,2023-01-01,NaN


In case the times are integers the frequency, start and end must also be integers.

In [14]:
df = pd.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': [2020, 2021, 2023, 2021, 2022],
        'y': np.arange(5),
    }
)
df

,unique_id,ds,y
0,0,2020,0
1,0,2021,1
2,0,2023,2
3,1,2021,3
4,1,2022,4


In [15]:
fill_gaps(
    df,
    freq=1,
    start=2019,
    end=2024,
)

,unique_id,ds,y
0,0,2019,NaN
1,0,2020,0.0
2,0,2021,1.0
3,0,2022,NaN
4,0,2023,2.0
5,0,2024,NaN
6,1,2019,NaN
7,1,2020,NaN
8,1,2021,3.0
9,1,2022,4.0


The function also accepts polars dataframes

In [16]:
#| polars
df = pl.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': [
            datetime(2020, 1, 1), datetime(2022, 1, 1), datetime(2023, 1, 1),
            datetime(2021, 1, 1), datetime(2022, 1, 1)],
        'y': np.arange(5),
    }
)
df

unique_id,ds,y
i64,datetime[μs],i64
0,2020-01-01 00:00:00,0
0,2022-01-01 00:00:00,1
0,2023-01-01 00:00:00,2
1,2021-01-01 00:00:00,3
1,2022-01-01 00:00:00,4


In [17]:
#| polars
polars_ms = fill_gaps(
    df.with_columns(pl.col('ds').cast(pl.Datetime(time_unit='ms'))),
    freq='1y',
    start=datetime(2019, 1, 1),
    end=datetime(2024, 1, 1),
)
assert polars_ms.schema['ds'].time_unit == 'ms'
polars_ms

unique_id,ds,y
i64,datetime[ms],i64
0,2019-01-01 00:00:00,null
0,2020-01-01 00:00:00,0
0,2021-01-01 00:00:00,null
0,2022-01-01 00:00:00,1
0,2023-01-01 00:00:00,2
…,…,…
1,2020-01-01 00:00:00,null
1,2021-01-01 00:00:00,3
1,2022-01-01 00:00:00,4


In [18]:
#| polars
df = pl.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': [
            date(2020, 1, 1), date(2022, 1, 1), date(2023, 1, 1),
            date(2021, 1, 1), date(2022, 1, 1)],
        'y': np.arange(5),
    }
)
df

unique_id,ds,y
i64,date,i64
0,2020-01-01,0
0,2022-01-01,1
0,2023-01-01,2
1,2021-01-01,3
1,2022-01-01,4


In [19]:
#| polars
fill_gaps(
    df,
    freq='1y',
    start=date(2020, 1, 1),
    end=date(2024, 1, 1),
)

unique_id,ds,y
i64,date,i64
0,2020-01-01,0
0,2021-01-01,null
0,2022-01-01,1
0,2023-01-01,2
0,2024-01-01,null
1,2020-01-01,null
1,2021-01-01,3
1,2022-01-01,4
1,2023-01-01,null


In [20]:
#| polars
df = pl.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': [2020, 2021, 2023, 2021, 2022],
        'y': np.arange(5),
    }
)
df

unique_id,ds,y
i64,i64,i64
0,2020,0
0,2021,1
0,2023,2
1,2021,3
1,2022,4


In [21]:
#| polars
fill_gaps(
    df,
    freq=1,
    start=2019,
    end=2024,
)

unique_id,ds,y
i64,i64,i64
0,2019,null
0,2020,0
0,2021,1
0,2022,null
0,2023,2
…,…,…
1,2020,null
1,2021,3
1,2022,4


In [22]:
#| hide
from itertools import product

from fastcore.test import test_warns

In [23]:
#| hide
def check_fill(dates, freq, start, end, include_start, include_end):
    base_idxs = []
    if include_start:
        base_idxs.append(0)
    if include_end:
        base_idxs.append(dates.size - 1)
    base_idxs = np.array(base_idxs, dtype=np.int64)
    date_idxs = np.hstack(
        [
            np.append(
                base_idxs,
                np.random.choice(
                    np.arange(1, dates.size - 1),
                    size=n_periods // 2 - len(base_idxs),
                    replace=False
                )
            ) for _ in range(2)
        ],
    )
    data = pd.DataFrame(
        {
            'unique_id': np.repeat([1, 2], n_periods // 2),
            'ds': dates[date_idxs],
            'y': np.arange(n_periods, dtype=np.float64)
        }
    )
    filled = fill_gaps(data, freq, start=start, end=end)
    data_starts_ends = data.groupby('unique_id', observed=True)['ds'].agg(['min', 'max'])
    global_start = data_starts_ends['min'].min()
    global_end = data_starts_ends['max'].max()
    filled_starts_ends = filled.groupby('unique_id', observed=True)['ds'].agg(['min', 'max'])

    # inferred frequency is the expected
    first_serie = filled[filled['unique_id'] == 1]
    if isinstance(freq, str):
        if first_serie['ds'].dt.tz is not None:
            first_serie = first_serie.copy()
            first_serie['ds'] = first_serie['ds'].dt.tz_convert('UTC')
        inferred_freq = pd.infer_freq(first_serie['ds'].dt.tz_localize(None))
        assert inferred_freq == pd.tseries.frequencies.to_offset(freq)
    else:
        assert all(first_serie['ds'].diff().value_counts().index == [freq])
    
    # fill keeps original data
    assert filled['y'].count() == n_periods
    # check starts
    if start == 'per_serie':
        pd.testing.assert_series_equal(
            data_starts_ends['min'], filled_starts_ends['min'],
        )        
    else: # global or specific
        min_dates = filled_starts_ends['min'].unique()
        assert min_dates.size == 1
        expected_start = global_start if start == 'global' else start
        assert min_dates[0] == expected_start

    # check ends
    if end == 'per_serie':
        pd.testing.assert_series_equal(
            data_starts_ends['max'], filled_starts_ends['max'],
        )        
    else: # global or specific
        max_dates = filled_starts_ends['max'].unique()
        assert max_dates.size == 1
        expected_end = global_end if end == 'global' else end
        assert max_dates[0] == expected_end

n_periods = 100
freqs = ['YE', 'YS', 'ME', 'MS', 'W', 'W-TUE', 'D', 's', 'ms', 1, 2, '20D', '30s', '2YE', '3YS', '30min', 'B', '1h', 'QS-NOV', 'QE']
try:
    pd.tseries.frequencies.to_offset('YE')
except ValueError:
    freqs = [
        f.replace('YE', 'Y')
        .replace('ME', 'M')
        .replace('h', 'H')
        .replace('QE', 'Q')
        for f in freqs if isinstance(f, str)
    ]
for freq in freqs:
    if isinstance(freq, (pd.offsets.BaseOffset, str)):
        offset = pd.tseries.frequencies.to_offset(freq)
        if isinstance(freq, str):
            try:
                delta = pd.Timedelta(freq)
                if delta.days > 0:
                    tz = None
                else:
                    tz = 'Europe/Berlin'
            except ValueError:
                tz = None
        dates = pd.date_range('1950-01-01', periods=n_periods, freq=freq, tz=tz)
    else:
        dates = np.arange(0, freq * n_periods, freq, dtype=np.int64)
        offset = freq
    global_start = dates[0]
    global_end = dates[-1]
    starts = ['global', 'per_serie', global_start - offset]
    ends = ['global', 'per_serie', global_end + offset]
    include_starts = [True, False]
    include_ends = [True, False]
    iterable = product(starts, ends, include_starts, include_ends)
    for start, end, include_start, include_end in iterable:
        check_fill(dates, freq, start, end, include_start, include_end)

In [24]:
#| hide
# last value doesn't meet frequency (year start)
dfx = pd.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': pd.to_datetime(['2020-01', '2021-01', '2023-01', '2021-01', '2022-02']),
        'y': np.arange(5),
    }
)
with warnings.catch_warnings(record=True) as w:
    fill_gaps(dfx, 'YS')
assert 'values were lost' in str(w[0].message)

In [25]:
#| hide
#| polars
# frequency and time column are not compatible
def error_freq(dates, freq, start, end, include_start, include_end, lib):
    base_idxs = []
    if include_start:
        base_idxs.append(0)
    if include_end:
        base_idxs.append(np.size(dates) - 1)
    base_idxs = np.array(base_idxs, dtype=np.int64)
    date_idxs = np.hstack(
        [
            np.append(
                base_idxs,
                np.random.choice(
                    np.arange(1, np.size(dates) - 1),
                    size=n_periods // 2 - len(base_idxs),
                    replace=False
                )
            ) for _ in range(2)
        ],
    )
    if lib == "pandas":
        data = pd.DataFrame(
            {
                'unique_id': np.repeat([1, 2], n_periods // 2),
                'ds': dates[date_idxs],
                'y': np.arange(n_periods, dtype=np.float64)
            }
        )
            
    if lib == "polars":
        data = pl.DataFrame(
            {
                'unique_id': np.repeat([1, 2], n_periods // 2),
                'ds': dates[date_idxs],
                'y': np.arange(n_periods, dtype=np.float64)
            }
        )

    try:
        filled = fill_gaps(data, freq, start=start, end=end)
    except Exception as e:
        assert isinstance(e, ValueError)


# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
freqs_pd = ['YE', 'YS', 'ME', 'MS', 'W', 'W-TUE', 'D', 's', 'ms', '20D', '30s', '2YE', '3YS', '30min', 'B', '1h', 'QS-NOV', 'QE']

# https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.Expr.dt.offset_by.html
freqs_pl = ['1d', '1w', '1mo', '1q', '1y']

# integer freqs
freqs_int = list(range(1, 10+1))

n_periods = 100
for lib in ['pandas', 'polars']:
    freqs_list = freqs_pd if lib == 'pandas' else freqs_pl
    for freq_int, freq_str in product(freqs_int, freqs_list):
        
        dates_int = np.arange(1, (n_periods*freq_int)+1, freq_int)

        
        if lib == 'pandas':
            dates_str = pd.date_range('1950-01-01', periods=n_periods, freq=freq_str)
            offset = pd.tseries.frequencies.to_offset(freq)
            first_date = dates_str[0] - offset
            last_date = dates_str[-1] + offset 
            
        if lib == 'polars':
            pl_dt = pl.date(1950,1,1)
            dates_str = pl.date_range(pl_dt, pl_dt.dt.offset_by(f"{n_periods}{freq_str[1:]}"), 
                                  interval=freq_str, eager=True)
            first_date = dates_str.dt.offset_by(f"-{freq_str}")[0]
            last_date = dates_str.dt.offset_by(freq_str)[-1]

        
        starts = ['global', 'per_serie', first_date]
        ends = ['global', 'per_serie', last_date]
        include_starts = [True, False]
        include_ends = [True, False]
        iterable = product(starts, ends, include_starts, include_ends)
        for start, end, include_start, include_end in iterable:
            error_freq(dates_str, freq_int, start, end, include_start, include_end, lib)
            error_freq(dates_int, freq_str, start, end, include_start, include_end, lib)